In [24]:
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
load_dotenv()

True

In [25]:
stone = pd.read_csv("../data/albumlist.csv", encoding='latin-1')

In [3]:
stone.sample(5)

,Number,Year,Album,Artist,Genre,Subgenre
100,101,1955,In the Wee Small Hours,Frank Sinatra,"Jazz, Pop","Big Band, Ballad"
352,353,2010,My Beautiful Dark Twisted Fantasy,Kanye West,Hip Hop,None
142,143,1968,Gris-Gris,"Dr. John, the Night Tripper","Jazz, Rock, Funk / Soul, Blues","Soul-Jazz, Louisiana Blues, Fusion, Bayou Funk"
284,285,1972,Music of My Mind,Stevie Wonder,Funk / Soul,"Soul-Jazz, Soul"
108,109,1966,Aftermath,The Rolling Stones,Rock,"Blues Rock, Pop Rock"


In [4]:
#stone.info()
stone.isna().sum() 

Number      0
Year        0
Album       0
Artist      0
Genre       0
Subgenre    0
dtype: int64

In [5]:
stone.Year.unique()

array([1967, 1966, 1965, 1971, 1972, 1979, 1968, 1976, 1959, 1969, 1975,
       1991, 1982, 1990, 1970, 1973, 1963, 1977, 1987, 2001, 1984, 1988,
       1957, 1964, 1956, 2000, 1986, 1980, 1978, 1955, 1962, 2003, 1995,
       1974, 2005, 1960, 1994, 1992, 1985, 2004, 1958, 1989, 1997, 2002,
       1998, 1983, 2006, 1981, 1961, 1999, 1993, 1996, 2007, 2010, 2011,
       2008])

In [6]:
stone.Album.unique()

array(["Sgt. Pepper's Lonely Hearts Club Band", 'Pet Sounds', 'Revolver',
       'Highway 61 Revisited', 'Rubber Soul', "What's Going On",
       'Exile on Main St.', 'London Calling', 'Blonde on Blonde',
       'The Beatles ("The White Album")', 'The Sun Sessions',
       'Kind of Blue', 'The Velvet Underground & Nico', 'Abbey Road',
       'Are You Experienced', 'Blood on the Tracks', 'Nevermind',
       'Born to Run', 'Astral Weeks', 'Thriller',
       'The Great Twenty_Eight', 'The Complete Recordings',
       'John Lennon/Plastic Ono Band', 'Innervisions',
       'Live at the Apollo, 1962', 'Rumours', 'The Joshua Tree',
       "Who's Next", 'Led Zeppelin', 'Blue', 'Bringing It All Back Home',
       'Let It Bleed', 'Ramones', 'Music From Big Pink',
       'The Rise and Fall of Ziggy Stardust and the Spiders From Mars',
       'Tapestry', 'Hotel California', 'The Anthology',
       'Please Please Me', 'Forever Changes',
       "Never Mind the Bollocks Here's the Sex Pistols", 'The 

In [26]:
stone.Album = stone.Album.str.replace("_","-")
stone.Album = stone.Album.str.replace("Ê"," ")
stone.Artist = stone.Artist.str.replace("Ê"," ")
stone.Genre = stone.Genre.str.replace("Ê"," ")
stone.Subgenre = stone.Subgenre.str.replace("Ê"," ")
stone.Artist = stone.Artist.str.replace("Wu Tang Clan","Wu-Tang Clan")
stone.Album = stone.Album.str.replace("Chteau","Chateau")
stone.Album = stone.Album.str.replace(r'(^Blues Breakers With Eric \w+.+)',"Blues Breakers",regex=True)
stone.Album = stone.Album.str.replace(r'(^The Band...The Brown.*)',"The Band",regex=True)
stone.Album = stone.Album.str.replace("The B 52's / Play Loud","The B-52's")
stone.Artist = stone.Artist.str.replace("The B 52's","The B-52's")
stone.Artist = stone.Artist.str.replace("Sleater Kinney","Sleater-Kinney")
stone.Album = stone.Album.str.replace("Deja vu","Déjà Vu")
#Déjà Vu
#The Beatles (Remastered)
#The Beatles ("The White Album")

stone.Album = stone.Album.str.replace("Proud Mary: The Best of Ike and Tina Turner","Best Of / Proud Mary")


In [46]:
#planets_2008 = planets[planets['year'] == 2008]
stone[stone["Artist"] == 'Sleater Kinney']

,Number,Year,Album,Artist,Genre,Subgenre,Alb_url,Gen,Subgen
271,272,1997,Dig Me Out,Sleater Kinney,Rock,Indie Rock,Dig Me Out,Rock,Indie Rock


In [27]:
stone["Alb_url"] = stone["Album"]

In [28]:
stone.Alb_url = stone.Alb_url.str.replace(r'(^Sign .Peace..the Times)',"Sign 'O' The Times",regex=True)
stone.Alb_url = stone.Alb_url.str.replace("#1 Record","%231 Record",regex=True)
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Beatles\s..The White Album..)',"The Beatles (Remastered)",regex=True)
stone.Alb_url = stone.Alb_url.str.replace("That's the Way of the World","That%27s the Way of the World",regex=True)

In [141]:
#stone[stone.Artist == "John Mayall & The Bluesbreakers"]["Album"] #194
#stone.iloc[194]['Album']

#stone.Album = stone.Album.str.replace('Blues Breakers With Eric Clapton ("The Beano A...',"Blues Breakers")
#stone.Album

In [142]:
stone[stone.Artist == "John Mayall & The Bluesbreakers"]["Album"]

194    Blues Breakers
Name: Album, dtype: object

In [30]:
stone

,Number,Year,Album,Artist,Genre,Subgenre,Alb_url
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock",Sgt. Pepper's Lonely Hearts Club Band
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock",Pet Sounds
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock",Revolver
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock",Highway 61 Revisited
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock,Rubber Soul
...,...,...,...,...,...,...,...
495,496,1969,Boz Scaggs,Boz Scaggs,Rock,Pop Rock,Boz Scaggs
496,497,2001,White Blood Cells,The White Stripes,Rock,"Indie Rock, Alternative Rock, Blues Rock, Gara...",White Blood Cells
497,498,1989,The Stone Roses,The Stone Roses,Rock,Indie Rock,The Stone Roses
498,499,1971,Live in Cook County Jail,B.B. King,Blues,Electric Blues,Live in Cook County Jail


In [143]:
stone[stone.Artist == "Elton John"]["Album"]
stone[stone.Artist == "John Mayall & The Bluesbreakers"]["Album"].index

Int64Index([194], dtype='int64')

In [144]:
stone.iloc[[20, 44, 92, 151, 194, 213, 358, 386, 433, 446]]

,Number,Year,Album,Artist,Genre,Subgenre,Alb_url
20,21,1982,The Great Twenty-Eight,Chuck Berry,Rock,Rock & Roll,The Great Twenty-Eight
44,45,1969,The Band,The Band,"Classical, Stage & Screen","Soundtrack, Modern Classical, Contemporary, Score",The Band
92,93,1987,"Sign ""Peace"" the Times",Prince,"Rock, Reggae","Ska, Reggae-Pop",Sign 'O' The Times
151,152,1979,The B-52's,The B 52's,"Electronic, Rock, Pop","New Wave, Punk, Mod",The B-52's
194,195,1966,Blues Breakers,John Mayall & The Bluesbreakers,"Rock, Blues","Blues Rock, Electric Blues, Harmonica Blues",Blues Breakers
213,214,1991,Best Of / Proud Mary,Ike & Tina Turner,Funk / Soul,"Rhythm & Blues, Bayou Funk, Soul",Best Of / Proud Mary
358,359,1972,Honky Chateau,Elton John,Rock,"Pop Rock, Classic Rock",Honky Chateau
386,387,1993,Enter the Wu-Tang: 36 Chambers,Wu-Tang Clan,Hip Hop,Gangsta,Enter the Wu-Tang: 36 Chambers
433,434,1972,#1 Record,Big Star,Rock,Power Pop,%231 Record
446,447,1964,Getz / Gilberto,Stan Getz / Joao Gilberto featuring Antonio Ca...,Jazz,"Bossa Nova, Latin Jazz",Getz / Gilberto


In [47]:
stone.Genre.value_counts().head(20)

Rock                            249
Funk / Soul                      38
Hip Hop                          29
Rock, Pop                        19
Electronic, Rock                 19
Rock, Blues                      17
Folk, World, & Country           13
Rock, Folk, World, & Country      9
Blues                             8
Electronic                        7
Jazz                              7
Reggae                            6
Funk / Soul, Pop                  6
Electronic, Pop                   5
Rock, Funk / Soul                 5
Funk / Soul, Blues                4
Hip Hop, Funk / Soul              3
Rock, Stage & Screen              3
Rock, Funk / Soul, Pop            3
Jazz, Pop                         3
Name: Genre, dtype: int64

In [37]:
stone["Gen"] = stone["Genre"].str.extract(r"(^([^,])+)")[0]

In [38]:
stone.Gen.value_counts()

Rock           318
Funk / Soul     51
Electronic      45
Hip Hop         34
Jazz            19
Folk            13
Blues            9
Reggae           7
Pop              2
Classical        1
Latin            1
Name: Gen, dtype: int64

In [39]:
stone["Subgen"] = stone["Subgenre"].str.extract(r"(^([^,])+)")[0]

In [40]:
stone.Subgen.unique()

array(['Rock & Roll', 'Pop Rock', 'Psychedelic Rock', 'Folk Rock', 'Soul',
       'Blues Rock', 'Punk', 'Modal', 'Garage Rock', 'Alternative Rock',
       'Acoustic', 'Disco', 'Delta Blues', 'Rhythm & Blues', 'Hard Rock',
       'Classic Rock', 'Folk', 'Beat', 'Prog Rock', 'Art Rock',
       'Soundtrack', 'Reggae', 'Free Jazz', 'Conscious', 'Electric Blues',
       'None', 'Dialogue', 'Doo Wop', 'Country', 'Ska', 'Fusion',
       'New Wave', 'Big Band', 'Hard Bop', 'Soft Rock', 'Funk',
       'Roots Reggae', 'Thug Rap', 'Indie Rock', 'Gangsta', 'Bayou Funk',
       'Chicago Blues', 'Soul-Jazz', 'Jazz-Rock', 'Afro-Cuban',
       'Post-Punk', 'Glam', 'Thrash', 'Power Pop', 'P.Funk', 'Europop',
       'Synth-pop', 'Country Rock', 'Acid Rock', 'Industrial', 'Grunge',
       'Piano Blues', 'Mod', 'Pop Rap', 'Country Blues',
       'Musique Concr?te', 'Heavy Metal', 'Electro', 'RnB/Swing', 'Vocal',
       'Noise', 'Cajun', 'Avantgarde', 'Modern Classical', 'Lo-Fi',
       'Contemporary Jazz'

In [41]:
stone["Subgen"] = stone["Subgen"].str.replace("Musique concrète ?te","Musique concrète", regex = True)

In [42]:
stone.Subgen.unique()

array(['Rock & Roll', 'Pop Rock', 'Psychedelic Rock', 'Folk Rock', 'Soul',
       'Blues Rock', 'Punk', 'Modal', 'Garage Rock', 'Alternative Rock',
       'Acoustic', 'Disco', 'Delta Blues', 'Rhythm & Blues', 'Hard Rock',
       'Classic Rock', 'Folk', 'Beat', 'Prog Rock', 'Art Rock',
       'Soundtrack', 'Reggae', 'Free Jazz', 'Conscious', 'Electric Blues',
       'None', 'Dialogue', 'Doo Wop', 'Country', 'Ska', 'Fusion',
       'New Wave', 'Big Band', 'Hard Bop', 'Soft Rock', 'Funk',
       'Roots Reggae', 'Thug Rap', 'Indie Rock', 'Gangsta', 'Bayou Funk',
       'Chicago Blues', 'Soul-Jazz', 'Jazz-Rock', 'Afro-Cuban',
       'Post-Punk', 'Glam', 'Thrash', 'Power Pop', 'P.Funk', 'Europop',
       'Synth-pop', 'Country Rock', 'Acid Rock', 'Industrial', 'Grunge',
       'Piano Blues', 'Mod', 'Pop Rap', 'Country Blues',
       'Musique Concr?te', 'Heavy Metal', 'Electro', 'RnB/Swing', 'Vocal',
       'Noise', 'Cajun', 'Avantgarde', 'Modern Classical', 'Lo-Fi',
       'Contemporary Jazz'

In [43]:
stone.Subgen.value_counts().head(5)

Pop Rock            43
Folk Rock           40
Alternative Rock    38
Blues Rock          37
None                29
Name: Subgen, dtype: int64

In [44]:
stone.sample(5)

,Number,Year,Album,Artist,Genre,Subgenre,Alb_url,Gen,Subgen
123,124,1967,Moby Grape,Moby Grape,Rock,"Folk Rock, Psychedelic Rock",Moby Grape,Rock,Folk Rock
321,322,1972,Sail Away,Randy Newman,"Rock, Pop",None,Sail Away,Rock,None
249,250,1996,Reasonable Doubt,Jay Z,Hip Hop,None,Reasonable Doubt,Hip Hop,None
200,201,1994,The Downward Spiral,Nine Inch Nails,"Electronic, Rock",Industrial,The Downward Spiral,Electronic,Industrial
15,16,1975,Blood on the Tracks,Bob Dylan,Rock,"Folk Rock, Acoustic, Ballad",Blood on the Tracks,Rock,Folk Rock


In [45]:
stone.to_csv("../data/stone.csv")